# 지식창고로 함수를 사용하는 방법

이 노트북에서는 지식창고에 액세스할 수 있는 에이전트와 사용자 요구 사항에 따라 호출할 수 있는 두 개의 함수를 만들어 [인수 생성]('How_to_generate_function_arguments_with_chat_models.ipynb') 노트북의 개념을 기반으로 합니다.

학술 주제에 대한 질문에 답하기 위해 arXiv의 데이터를 사용하는 에이전트를 만들어 보겠습니다. 이 에이전트에는 다음과 같은 두 가지 함수가 있습니다: 

- **get_articles:** 주제에 대한 arXiv 논문을 가져와 링크와 함께 사용자에게 요약하는 함수 
- **read_article_and_summarize**: 이 기능은 이전에 검색한 논문 중 하나를 가져와 전체 내용을 읽고 핵심 주장, 증거 및 결론을 요약합니다.

이렇게 하면 여러 서비스 중에서 선택할 수 있고 첫 번째 함수의 데이터 중 일부가 두 번째 함수에서 계속 사용되는 다기능 워크플로우에 익숙해질 수 있습니다.

## 연습

이 쿡북에서는 다음과 같은 워크플로우를 안내합니다:

- **검색 유틸리티:** 답변을 얻기 위해 arXiv에 액세스하는 두 가지 함수 만들기 
- **에이전트 구성:** 함수의 필요성을 평가하고, 필요한 경우 해당 함수를 호출하여 결과를 에이전트에게 다시 제시하는 에이전트 동작 구축 
- **arXiv 대화:** 이 모든 것을 실시간 대화에 통합하기.

In [1]:
!pip install scipy
!pip install tenacity
!pip install tiktoken==0.3.3
!pip install termcolor 
!pip install openai
!pip install requests
!pip install arxiv
!pip install pandas
!pip install PyPDF2
!pip install tqdm

In [2]:
import arxiv
import ast
import concurrent
from csv import writer
from IPython.display import display, Markdown, Latex
import json
import openai
import os
import pandas as pd
from PyPDF2 import PdfReader
import requests
from scipy import spatial
from tenacity import retry, wait_random_exponential, stop_after_attempt
import tiktoken
from tqdm import tqdm
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-16k-0613"
# GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"


## 검색 유틸리티

먼저 두 가지 기능을 뒷받침할 몇 가지 유틸리티를 설정하겠습니다.

다운로드한 논문은 디렉토리에 저장됩니다(여기서는 ```./data/papers```을 사용합니다). 다운로드한 논문의 임베딩과 세부 정보를 저장하여 ```summarize_text```를 사용하여 검색할 수 있도록 ``arxiv_library.csv`` 파일을 생성합니다.

In [3]:
# Set a directory to store downloaded papers
data_dir = os.path.join(os.curdir, "data", "papers")
paper_dir_filepath = "./data/arxiv_library.csv"

# Generate a blank dataframe where we can store downloaded files
df = pd.DataFrame(list())
df.to_csv(paper_dir_filepath)


In [4]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def embedding_request(text):
    response = openai.Embedding.create(input=text, model=EMBEDDING_MODEL)
    return response


def get_articles(query, library=paper_dir_filepath, top_k=5):
    """This function gets the top_k articles based on a user's query, sorted by relevance.
    It also downloads the files and stores them in arxiv_library.csv to be retrieved by the read_article_and_summarize.
    """
    search = arxiv.Search(
        query=query, max_results=top_k, sort_by=arxiv.SortCriterion.Relevance
    )
    result_list = []
    for result in search.results():
        result_dict = {}
        result_dict.update({"title": result.title})
        result_dict.update({"summary": result.summary})

        # Taking the first url provided
        result_dict.update({"article_url": [x.href for x in result.links][0]})
        result_dict.update({"pdf_url": [x.href for x in result.links][1]})
        result_list.append(result_dict)

        # Store references in library file
        response = embedding_request(text=result.title)
        file_reference = [
            result.title,
            result.download_pdf(data_dir),
            response["data"][0]["embedding"],
        ]

        # Write to file
        with open(library, "a") as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(file_reference)
            f_object.close()
    return result_list


In [5]:
# Test that the search is working
result_output = get_articles("ppo reinforcement learning")
result_output[0]


{'title': 'Proximal Policy Optimization and its Dynamic Version for Sequence Generation',
 'summary': 'In sequence generation task, many works use policy gradient for model\noptimization to tackle the intractable backpropagation issue when maximizing\nthe non-differentiable evaluation metrics or fooling the discriminator in\nadversarial learning. In this paper, we replace policy gradient with proximal\npolicy optimization (PPO), which is a proved more efficient reinforcement\nlearning algorithm, and propose a dynamic approach for PPO (PPO-dynamic). We\ndemonstrate the efficacy of PPO and PPO-dynamic on conditional sequence\ngeneration tasks including synthetic experiment and chit-chat chatbot. The\nresults show that PPO and PPO-dynamic can beat policy gradient by stability and\nperformance.',
 'article_url': 'http://arxiv.org/abs/1808.07982v1',
 'pdf_url': 'http://arxiv.org/pdf/1808.07982v1'}

In [6]:
result_output

[{'title': 'Proximal Policy Optimization and its Dynamic Version for Sequence Generation',
  'summary': 'In sequence generation task, many works use policy gradient for model\noptimization to tackle the intractable backpropagation issue when maximizing\nthe non-differentiable evaluation metrics or fooling the discriminator in\nadversarial learning. In this paper, we replace policy gradient with proximal\npolicy optimization (PPO), which is a proved more efficient reinforcement\nlearning algorithm, and propose a dynamic approach for PPO (PPO-dynamic). We\ndemonstrate the efficacy of PPO and PPO-dynamic on conditional sequence\ngeneration tasks including synthetic experiment and chit-chat chatbot. The\nresults show that PPO and PPO-dynamic can beat policy gradient by stability and\nperformance.',
  'article_url': 'http://arxiv.org/abs/1808.07982v1',
  'pdf_url': 'http://arxiv.org/pdf/1808.07982v1'},
 {'title': 'CIM-PPO:Proximal Policy Optimization with Liu-Correntropy Induced Metric',
  

In [7]:
# Test that the search is working
result_output = get_articles("toolformer")
result_output[0]


{'title': 'Graph-ToolFormer: To Empower LLMs with Graph Reasoning Ability via Prompt Augmented by ChatGPT',
 'summary': 'In this paper, we aim to develop a large language model (LLM) with the\nreasoning ability on complex graph data. Currently, LLMs have achieved very\nimpressive performance on various natural language learning tasks, extensions\nof which have also been applied to study the vision tasks with multi-modal\ndata. However, when it comes to the graph learning tasks, existing LLMs present\nvery serious flaws due to their several inherited weaknesses in performing\n{multi-step logic reasoning}, {precise mathematical calculation} and\n{perception about the spatial and temporal factors}.\n  To address such challenges, in this paper, we will investigate the\nprinciples, methodologies and algorithms to empower existing LLMs with graph\nreasoning ability, which will have tremendous impacts on the current research\nof both LLMs and graph learning. Inspired by the latest ChatGPT and

In [8]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100,
) -> list[str]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = embedding_request(query)
    query_embedding = query_embedding_response["data"][0]["embedding"]
    strings_and_relatednesses = [
        (row["filepath"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n]


In [9]:
def read_pdf(filepath):
    """Takes a filepath to a PDF and returns a string of the PDF's contents"""
    # creating a pdf reader object
    reader = PdfReader(filepath)
    pdf_text = ""
    page_number = 0
    for page in reader.pages:
        page_number += 1
        pdf_text += page.extract_text() + f"\nPage Number: {page_number}"
    return pdf_text


# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    """Returns successive n-sized chunks from provided text."""
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.5 * n and 1.5 * n tokens
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j


def extract_chunk(content, template_prompt):
    """This function applies a prompt to some input content. In this case it returns a summarize chunk of text"""
    prompt = template_prompt + content
    response = openai.ChatCompletion.create(
        model=GPT_MODEL, messages=[{"role": "user", "content": prompt}], temperature=0
    )
    return response["choices"][0]["message"]["content"]


def summarize_text(query):
    """This function does the following:
    - Reads in the arxiv_library.csv file in including the embeddings
    - Finds the closest file to the user's query
    - Scrapes the text out of the file and chunks it
    - Summarizes each chunk in parallel
    - Does one final summary and returns this to the user"""

    # A prompt to dictate how the recursive summarizations should approach the input paper
    summary_prompt = """Summarize this text from an academic paper. Extract any key points with reasoning.\n\nContent:"""

    # If the library is empty (no searches have been performed yet), we perform one and download the results
    library_df = pd.read_csv(paper_dir_filepath).reset_index()
    if len(library_df) == 0:
        print("No papers searched yet, downloading first.")
        get_articles(query)
        print("Papers downloaded, continuing")
        library_df = pd.read_csv(paper_dir_filepath).reset_index()
    library_df.columns = ["title", "filepath", "embedding"]
    library_df["embedding"] = library_df["embedding"].apply(ast.literal_eval)
    strings = strings_ranked_by_relatedness(query, library_df, top_n=1)
    print("Chunking text from paper")
    pdf_text = read_pdf(strings[0])

    # Initialise tokenizer
    tokenizer = tiktoken.get_encoding("cl100k_base")
    results = ""

    # Chunk up the document into 1500 token chunks
    chunks = create_chunks(pdf_text, 1500, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    print("Summarizing each chunk of text")

    # Parallel process the summaries
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=len(text_chunks)
    ) as executor:
        futures = [
            executor.submit(extract_chunk, chunk, summary_prompt)
            for chunk in text_chunks
        ]
        with tqdm(total=len(text_chunks)) as pbar:
            for _ in concurrent.futures.as_completed(futures):
                pbar.update(1)
        for future in futures:
            data = future.result()
            results += data

    # Final summary
    print("Summarizing into overall summary")
    response = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[
            {
                "role": "user",
                "content": f"""Write a summary in Korean collated from this collection of key points extracted from an academic paper. Must use Korean.
                        The summary should highlight the core argument, conclusions and evidence, and answer the user's query.
                        User query: {query}
                        The summary should be structured in bulleted lists following the headings Core Argument, Evidence, and Conclusions.
                        Key points:\n{results}\nSummary:\n""",
            }
        ],
        temperature=0,
    )
    return response


In [10]:
# Test the summarize_text function works
chat_test_response = summarize_text("PPO reinforcement learning sequence generation")


Chunking text from paper
Summarizing each chunk of text


100%|██████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.45s/it]


Summarizing into overall summary


In [11]:
print(chat_test_response["choices"][0]["message"]["content"])


학술 논문은 시퀀스 생성 작업에서 Proximal Policy Optimization (PPO)의 사용에 대해 논의하고, 특히 챗봇의 대화 작업에서의 사용에 초점을 맞추고 있다. 저자들은 텍스트 생성 작업에서 일반적으로 사용되는 정책 기울기와 비교하여 PPO가 보다 효율적인 강화 학습 알고리즘이라고 주장한다. 저자들은 PPO의 동적 접근법 (PPO-dynamic)을 제안하고, 합성 실험과 챗봇 작업에서의 효과를 입증한다. 결과는 PPO와 PPO-dynamic이 안정성과 성능 측면에서 정책 기울기를 능가한다는 것을 보여준다. 이 논문은 또한 정책 기울기와 PPO에 대한 배경 정보를 제공하며, 텍스트 생성 분야에서의 관련 연구를 논의한다.

핵심 주장:
- PPO는 시퀀스 생성 작업에서 정책 기울기보다 더 효율적인 강화 학습 알고리즘이다.
- PPO-dynamic은 PPO의 하이퍼파라미터를 동적으로 조정하여 최적화 과정을 개선한다.
- PPO와 PPO-dynamic은 안정성과 성능 측면에서 정책 기울기를 능가한다.

증거:
- 합성 카운팅 작업에서 PPO-dynamic은 REINFORCE와 MIXER와 비교하여 높은 정확도 점수를 달성한다.
- PPO-dynamic의 학습 곡선은 PPO보다 빠른 진전을 보인다.
- 챗봇 작업에서 PPO-dynamic은 REINFORCE와 PPO보다 약간 높은 BLEU-2 점수를 달성한다.
- PPO-dynamic의 학습 곡선은 정책 기울기보다 안정적이고 빠르다.

결론:
- PPO는 정책 기울기에 비해 시퀀스 학습에 더 나은 최적화 방법이다.
- PPO-dynamic은 하이퍼파라미터를 동적으로 조정하여 최적화 과정을 개선한다.
- 저자들은 PPO를 GAN 기반 시퀀스 학습의 새로운 최적화 방법으로 사용할 수 있으며, 이는 더 나은 성능을 제공한다고 결론 짓는다.


## 에이전트 구성

이 단계에서는 API를 여러 차례 사용할 수 있도록 지원하는 ```Conversation``` 클래스와 ```ChatCompletion``` API와 지식창고 함수 간의 상호 작용을 지원하는 몇 가지 Python 함수를 포함하여 에이전트를 만들겠습니다.

In [12]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [13]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [14]:
# Initiate our get_articles and read_article_and_summarize functions
arxiv_functions = [
    {
        "name": "get_articles",
        "description": """Use this function to get academic papers from arXiv to answer user questions.""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            User query in JSON. Responses should be summarized and should include the article URL reference
                            """,
                }
            },
            "required": ["query"],
        },
        "name": "read_article_and_summarize",
        "description": """Use this function to read whole papers and provide a summary for users.
        You should NEVER call this function before get_articles has been called in the conversation.""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            Description of the article in plain text based on the user's query
                            """,
                }
            },
            "required": ["query"],
        },
    }
]


In [15]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """This function makes a ChatCompletion API call with the option of adding functions"""
    response = chat_completion_request(messages, functions)
    full_message = response.json()["choices"][0]
    if full_message["finish_reason"] == "function_call":
        print(f"Function generation requested, calling function")
        return call_arxiv_function(messages, full_message)
    else:
        print(f"Function not required, responding to user")
        return response.json()


def call_arxiv_function(messages, full_message):
    """Function calling function which executes function calls when the model believes it is necessary.
    Currently extended by adding clauses to this if statement."""

    if full_message["message"]["function_call"]["name"] == "get_articles":
        try:
            parsed_output = json.loads(
                full_message["message"]["function_call"]["arguments"]
            )
            print("Getting search results")
            results = get_articles(parsed_output["query"])
        except Exception as e:
            print(parsed_output)
            print(f"Function execution failed")
            print(f"Error message: {e}")
        messages.append(
            {
                "role": "function",
                "name": full_message["message"]["function_call"]["name"],
                "content": str(results),
            }
        )
        try:
            print("Got search results, summarizing content")
            response = chat_completion_request(messages)
            return response.json()
        except Exception as e:
            print(type(e))
            raise Exception("Function chat request failed")

    elif (
        full_message["message"]["function_call"]["name"] == "read_article_and_summarize"
    ):
        parsed_output = json.loads(
            full_message["message"]["function_call"]["arguments"]
        )
        print("Finding and reading paper")
        summary = summarize_text(parsed_output["query"])
        return summary

    else:
        raise Exception("Function does not exist and cannot be called")


## arXiv 대화

대화에서 함수를 테스트하여 이 모든 것을 종합해 보겠습니다.

In [16]:
# Start with a system message
paper_system_message = """You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.
You summarize the papers clearly so the customer can decide which to read to answer their question.
You always provide the article_url and title so the user can understand the name of the paper and click through to access it.
Begin!"""
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)


In [17]:
# Add a user message
paper_conversation.add_message("user", "Hi, how does PPO reinforcement learning work?")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))


Function generation requested, calling function
Finding and reading paper
Chunking text from paper
Summarizing each chunk of text


100%|████████████████████████████████████████████████████████████████████████| 17/17 [00:32<00:00,  1.89s/it]


ServiceUnavailableError: The server is overloaded or not ready yet.

In [18]:
# Add another user message to induce our system to use the second tool
paper_conversation.add_message(
    "user",
    "Can you read the PPO sequence generation paper for me and give me a summary",
)
updated_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)
display(Markdown(updated_response["choices"][0]["message"]["content"]))


Function generation requested, calling function
Finding and reading paper
Chunking text from paper
Summarizing each chunk of text


100%|██████████████████████████████████████████████████████████████████████████| 4/4 [00:10<00:00,  2.55s/it]


Summarizing into overall summary


핵심 주장:
- 이 논문은 시퀀스 생성 작업에서 Proximal Policy Optimization (PPO)의 사용에 대해 논의한다.
- PPO-dynamic이라는 동적 접근법을 제안하고, 이를 합성 실험과 chit-chat 챗봇 작업에서의 효과를 보여준다.
- 결과는 PPO와 PPO-dynamic이 안정성과 성능 측면에서 policy gradient보다 우수함을 보여준다.

증거:
- PPO-dynamic은 합성 계수 작업에서 REINFORCE와 MIXER와 비교했을 때 높은 정확도 점수를 달성한다.
- PPO-dynamic의 학습 곡선은 PPO보다 빠른 진전을 보인다.
- chit-chat 챗봇 작업에서 PPO-dynamic은 REINFORCE와 PPO보다 약간 높은 BLEU-2 점수를 달성한다.
- PPO와 PPO-dynamic의 학습 곡선은 policy gradient보다 안정적이며, PPO-dynamic은 더 빠르게 수렴한다.

결론:
- PPO는 시퀀스 학습에 있어서 policy gradient보다 더 나은 최적화 방법이다.
- PPO-dynamic은 하이퍼파라미터를 동적으로 조정하여 최적화 과정을 개선한다.
- PPO는 GAN 기반 시퀀스 학습을 위한 새로운 최적화 방법으로 사용될 수 있다.

In [19]:
paper_conversation.conversation_history

[{'role': 'system',
  'content': 'You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.\nYou summarize the papers clearly so the customer can decide which to read to answer their question.\nYou always provide the article_url and title so the user can understand the name of the paper and click through to access it.\nBegin!'},
 {'role': 'user', 'content': 'Hi, how does PPO reinforcement learning work?'},
 {'role': 'user',
  'content': 'Can you read the PPO sequence generation paper for me and give me a summary'}]

In [20]:
# Start with a system message
paper_system_message = """You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.
You summarize the papers clearly so the customer can decide which to read to answer their question.
You always provide the article_url and title so the user can understand the name of the paper and click through to access it.
Begin!"""
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)


In [21]:
paper_conversation.conversation_history

[{'role': 'system',
  'content': 'You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.\nYou summarize the papers clearly so the customer can decide which to read to answer their question.\nYou always provide the article_url and title so the user can understand the name of the paper and click through to access it.\nBegin!'}]

## transformer 논문

In [22]:
# Start with a system message
paper_system_message = """You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.
You summarize the papers clearly so the customer can decide which to read to answer their question.
You always provide the article_url and title so the user can understand the name of the paper and click through to access it.
Begin!"""
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)


In [23]:
paper_conversation.conversation_history

[{'role': 'system',
  'content': 'You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.\nYou summarize the papers clearly so the customer can decide which to read to answer their question.\nYou always provide the article_url and title so the user can understand the name of the paper and click through to access it.\nBegin!'}]

In [24]:
# Add a user message
paper_conversation.add_message("user", "Hi, how does Transformer work?")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))


Function generation requested, calling function
Finding and reading paper
Chunking text from paper
Summarizing each chunk of text


100%|████████████████████████████████████████████████████████████████████████| 10/10 [00:12<00:00,  1.29s/it]


Summarizing into overall summary


Toolformer is a language model that can teach itself to use external tools via simple APIs. It overcomes the limitations of current language models by accessing up-to-date information and performing precise calculations. The model is trained in a self-supervised way, incorporating a range of tools such as a calculator, Q&A system, search engine, translation system, and calendar. The key steps of the approach involve sampling, executing, and filtering API calls, and merging them into the LM dataset. Toolformer achieves improved zero-shot performance on various tasks without sacrificing its core language modeling abilities. The paper also discusses the tools used in experiments and evaluates the model's performance in zero-shot settings.